In [1]:
!conda install pytorch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 pytorch-cuda=11.8 -c pytorch -c nvidia -y

Channels:
 - pytorch
 - nvidia
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Read timed out. (read timeout=9.15)")': /pkgs/msys2/win-64/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Read timed out. (read timeout=9.15)")': /pkgs/main/win-64/repodata.json.zst

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Read timed out. (read timeout=60.0)")': /pkgs/main/noarch/repodata.json.zst



In [2]:
import torch

In [3]:
#파이토치 GPU 사용 여부 확인
torch.cuda.is_available()

True

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
import numpy as np
import pickle
from tqdm import tqdm
import random
import math
from tqdm.notebook import tqdm

In [5]:
#word_to_index.pickle : 수어 스크립트 단어를 어떤 숫자로 변환 해야하는지 저장한 파일
with open('data/word_to_index.pickle', 'rb') as f:
    #수어 스크립트 단어를 어떤 숫자로 변환 해야하는지 저장한 파일 읽어서 word_to_index에 저장
    word_to_index = pickle.load(f)

In [6]:
word_to_index

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 '6분': 3,
 '깊고': 4,
 '고흥군': 5,
 '되었을': 6,
 '빼고': 7,
 '20분': 8,
 '위경련': 9,
 '뒤집': 10,
 '벗고': 11,
 '털어': 12,
 '지탱': 13,
 '휴식': 14,
 '우리': 15,
 '용문산': 16,
 '35': 17,
 '1시': 18,
 '된': 19,
 '14': 20,
 '영등포구': 21,
 '바닥': 22,
 '지키시기': 23,
 '되었습니다': 24,
 '농사일': 25,
 '도크': 26,
 '식수': 27,
 '별': 28,
 '위험하므로': 29,
 '건물': 30,
 '세종': 31,
 '서': 32,
 '겼을': 33,
 '도내': 34,
 '물이': 35,
 '생긴': 36,
 '화악산': 37,
 '어려울': 38,
 '남부': 39,
 '14일': 40,
 '어깨': 41,
 '강변': 42,
 '그러한': 43,
 '때': 44,
 '05일': 45,
 '으로': 46,
 '실외': 47,
 '라디오': 48,
 '5월': 49,
 '여유': 50,
 '백운산': 51,
 '반': 52,
 '여주시': 53,
 '검은': 54,
 '나': 55,
 '완만': 56,
 '지날': 57,
 '시청': 58,
 '오후': 59,
 '북동쪽': 60,
 '15일': 61,
 '전라남도': 62,
 '야': 63,
 '건넌다': 64,
 '근육': 65,
 '뒤쪽': 66,
 '우천': 67,
 '청': 68,
 '낮게하고': 69,
 '로프': 70,
 '놀라서': 71,
 '경주': 72,
 '자': 73,
 '거북이': 74,
 '암벽': 75,
 '건너지': 76,
 '밀고': 77,
 '끼': 78,
 '달리는': 79,
 '사람': 80,
 '이나': 81,
 '물기': 82,
 '하천': 83,
 '맡기고': 84,
 '바랍니다': 85,
 '남남서': 86,
 '경기': 87,
 '

In [7]:
#len(word_to_index) : 저장된 단어의 개수
len(word_to_index)

1078

In [8]:
HID_DIM = 512
OUTPUT_DIM = len(word_to_index)
N_LAYER = 2 
DEC_DROPOUT = 0.5
EMB_DIM = 128
BATCH_SIZE = 32
N_EPOCH = 1000
CLIP = 1
learning_rate = 0.0001
model_save_path = 'data/'
save_model_name = 'model1.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cuda:0


In [9]:
#np.load('data/X_data.npy') : keypoint가 저장죈 X_data.npy 읽어서 리턴
X_data = np.load('data/X_data.npy')

In [10]:
X_data.shape

(352, 240, 134)

In [11]:
X_data[0:3]

array([[[ 0.33139095, -1.2248869 ,  0.31375602, ...,  0.59203297,
         -0.07408668,  0.59665173],
        [ 0.33017254, -1.2249231 ,  0.31251508, ...,  0.593432  ,
         -0.07786492,  0.5957069 ],
        [ 0.32986647, -1.2254999 ,  0.3121397 , ...,  0.5932884 ,
         -0.07602021,  0.5910137 ],
        ...,
        [ 0.34666264, -1.2243315 ,  0.32900348, ...,  0.57456756,
         -0.10940002,  0.5609939 ],
        [ 0.34107053, -1.224629  ,  0.3319686 , ...,  0.5701985 ,
         -0.10562919,  0.5633026 ],
        [ 0.34480435, -1.2247169 ,  0.33569857, ...,  0.567693  ,
         -0.10210186,  0.5561903 ]],

       [[ 0.33823052, -1.2373524 ,  0.3206036 , ...,  0.5847284 ,
         -0.07379816,  0.5847284 ],
        [ 0.33649135, -1.2290341 ,  0.3270037 , ...,  0.58165026,
         -0.0718272 ,  0.59097505],
        [ 0.34267113, -1.2379397 ,  0.32487038, ...,  0.5861447 ,
         -0.07286945,  0.5861447 ],
        ...,
        [ 0.3576922 , -1.2186489 ,  0.31385562, ...,  

In [12]:
#np.load('data/y_data.npy) : 수어 스크립트가 저장된 y_data.npy 읽어서 리턴
y_data = np.load('data/y_data.npy')

In [13]:
y_data[0:3]

array([[  1, 103, 736, 720, 202, 954, 337, 226, 498, 916, 847, 900, 662,
        757, 123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  1, 505, 305, 818, 916, 954, 337, 226, 202, 498, 847, 900, 662,
        757, 123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  1, 484, 448, 994, 954, 337, 226, 498, 498, 847, 900, 662, 757,
        123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [14]:
y_data = torch.tensor(y_data)

In [15]:
y_data.shape

torch.Size([352, 30])

In [16]:
X_data = torch.tensor(X_data)

In [17]:
X_data.shape

torch.Size([352, 240, 134])

In [18]:
#D.TensorDataset(X_data,y_data): X_data와 y_data를 리턴할 객체 생성
train_dataset = D.TensorDataset(X_data, y_data)
train_dataset

In [19]:
#torch.utils.data.DataLoader( 
#                            train_dataset
#                            , batch_size=16 : 학습시 16행씩 리턴
#                            , shuffle=True : X_data 와 y_data의 행을 섞음
#                            , drop_last=False : 마지막 데이터의 행이 16개 미만일때도 학습 진행
#                            )
# : 학습시 X_data 와 y_data 를 리턴하는 객체 생성
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=False)

In [20]:
train_dataloader

In [21]:
for index, (inp,ta) in enumerate(train_dataloader):
    print("index=",index)
    print("inp=",inp.shape)
    print("ta=",ta.shape)
    print("="*100)
    break

index= 0
inp= torch.Size([16, 240, 134])
ta= torch.Size([16, 30])


In [22]:
#입력데이터
#정지화면 1개당 134개의 키포인트가 존재
#240개의 정지 화면으로 구성
X_data[0:1]

tensor([[[ 0.3314, -1.2249,  0.3138,  ...,  0.5920, -0.0741,  0.5967],
         [ 0.3302, -1.2249,  0.3125,  ...,  0.5934, -0.0779,  0.5957],
         [ 0.3299, -1.2255,  0.3121,  ...,  0.5933, -0.0760,  0.5910],
         ...,
         [ 0.3467, -1.2243,  0.3290,  ...,  0.5746, -0.1094,  0.5610],
         [ 0.3411, -1.2246,  0.3320,  ...,  0.5702, -0.1056,  0.5633],
         [ 0.3448, -1.2247,  0.3357,  ...,  0.5677, -0.1021,  0.5562]]])

In [23]:
X_data[0:1].shape

torch.Size([1, 240, 134])

In [24]:
#X_data.shape[2] : X_data의 열의 개수 리턴
input_size = X_data.shape[2]

In [25]:
input_size

134

In [26]:
#X_data.shape[2] : X_data의 데이터의 수를 리턴
x_row = X_data[0:1].shape[0]
x_row

1

In [27]:
#N_LAYER =2
#HID_DIM = 512
#n_row = 1

#torch.zeros(shape) : 0으로 초기화된 배열 생성
#torch.zeros(N_LAYER *2, x_row, HID_DIM) : [4,1,512] shape 0으로 초기화된 3차원 배열 생성

#cuda() : GPU에 배열 저장

#float() : 배열 타입을 실수로 설정
h0 = torch.zeros(N_LAYER *2, x_row, HID_DIM).cuda().float()

In [28]:
h0

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

In [29]:
h0.shape

torch.Size([4, 1, 512])

In [30]:
input_size

134

In [31]:
HID_DIM

512

In [32]:
gru = nn.GRU(
    input_size, #입력되는 keypoint 1개의 개수 (134개의 keypoint가 순차적으로 입력됨
    HID_DIM, #hidden layer의 size HID_DIM = 512
    N_LAYER, # N_LAYER = 2 2개의 GRU가 쌓여서 실행
    batch_first = True, #계산 결과가 [데이터수, 입력데이터행, 입력데이터열] 리턴 => [1,240,1024]
    bidirectional = True,
    dropout=DEC_DROPOUT #Dropout 설정
).cuda()

In [33]:
gru

GRU(134, 512, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)

In [34]:
#GRU 연산 실행
encoder_output, encoder_hidden = gru(X_data[0:1].cuda() , h0.cuda())

In [35]:
encoder_output

tensor([[[ 0.1505,  0.2497,  0.0811,  ..., -0.1775,  0.1487,  0.0590],
         [ 0.1277,  0.1981,  0.3587,  ..., -0.1642,  0.1019, -0.0024],
         [ 0.0820,  0.1381,  0.3671,  ..., -0.1341, -0.0537,  0.0109],
         ...,
         [-0.1311,  0.1731,  0.3574,  ..., -0.0007, -0.1417,  0.0567],
         [-0.1756,  0.2728,  0.1529,  ...,  0.0329, -0.2524,  0.0442],
         [-0.1394,  0.3053,  0.1561,  ..., -0.0814, -0.2073, -0.0540]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward0>)

In [36]:
encoder_output.shape

torch.Size([1, 240, 1024])

In [37]:
encoder_hidden

tensor([[[ 0.4200, -0.5739, -0.0171,  ...,  0.4684,  0.1007, -0.4054]],

        [[ 0.0485,  0.2034,  0.0958,  ..., -0.0015, -0.0476,  0.0534]],

        [[-0.1394,  0.3053,  0.1561,  ...,  0.4225, -0.0656, -0.0832]],

        [[-0.0410,  0.2059,  0.4497,  ..., -0.1775,  0.1487,  0.0590]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward0>)

In [38]:
encoder_hidden.shape

torch.Size([4, 1, 512])

In [39]:
encoder_hidden.shape

torch.Size([4, 1, 512])

In [40]:
#
# encoder_hidden[-4,:,:],
# encoder_hidden[-3,:,:],
# encoder_hidden[-2,:,:],
# encoder_hidden[-1,:,:]
#을 1개로 합쳐서 2차원 배열로 변환

#dim = 1 : 배열의 1천뺴 차원에서 합침

torch.cat((
     encoder_hidden[-4,:,:],
     encoder_hidden[-3,:,:],
     encoder_hidden[-2,:,:],
     encoder_hidden[-1,:,:]
), dim = 1)

tensor([[ 0.4200, -0.5739, -0.0171,  ..., -0.1775,  0.1487,  0.0590]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [41]:
#
# encoder_hidden[-4,:,:],
# encoder_hidden[-3,:,:],
# encoder_hidden[-2,:,:],
# encoder_hidden[-1,:,:]
#을 1개로 합쳐서 2차원 배열로 변환

#dim = 1 : 배열의 1천뺴 차원에서 합침

encoder_hidden = torch.cat((
     encoder_hidden[-4,:,:],
     encoder_hidden[-3,:,:],
     encoder_hidden[-2,:,:],
     encoder_hidden[-1,:,:]
), dim = 1)

In [42]:
encoder_hidden.shape

torch.Size([1, 2048])

In [43]:
#선형 회귀 객체 생성
fc = nn.Linear(
    HID_DIM * 4, #입력데이터 (encoder_hidden)의 열의 개수 HID_DIM (512)
    HID_DIM #계상 결과 (추력데이터)의 열의 개수 HID_DIM (512)
).cuda()

In [44]:
encoder_hidden = fc(encoder_hidden)

In [45]:
encoder_hidden

tensor([[-2.6731e-02, -2.8492e-02, -2.2107e-02, -7.6137e-02, -1.6818e-01,
          4.1211e-02,  1.1476e-01, -2.2332e-01, -9.6641e-02, -2.7740e-02,
          9.9955e-02, -2.4105e-01,  4.3593e-02, -4.6121e-03, -1.9914e-01,
          4.8469e-02,  4.3703e-02,  1.6191e-01,  7.6041e-02, -1.1429e-01,
          4.9460e-02, -2.8985e-01,  2.1878e-01, -1.7574e-01, -7.1943e-02,
          4.2752e-03,  1.3126e-01, -4.7563e-02, -1.6651e-01,  9.5412e-03,
          1.3457e-01, -1.0672e-02,  3.6466e-01,  3.8801e-01,  8.3215e-02,
         -7.3927e-02,  2.4566e-01,  5.6862e-02,  2.1404e-01, -1.0948e-01,
         -1.6038e-01, -1.1524e-01,  2.9680e-01,  7.4328e-02,  6.4268e-02,
          1.4260e-01, -1.0181e-01,  9.1755e-02, -1.8662e-02, -2.6620e-02,
          1.0658e-02,  2.9316e-01,  2.1279e-02,  1.0248e-02, -3.9928e-03,
         -3.7584e-04,  1.2952e-02,  2.6277e-01,  4.2112e-02,  2.1225e-01,
          7.5519e-02, -7.3691e-02, -9.6566e-02, -5.2502e-02, -3.8493e-03,
          9.0256e-03,  2.4548e-01, -2.

In [46]:
encoder_hidden.shape

torch.Size([1, 512])

In [47]:
#활성함수 tanh 실행

torch.tanh(encoder_hidden)

tensor([[-2.6725e-02, -2.8484e-02, -2.2103e-02, -7.5990e-02, -1.6661e-01,
          4.1188e-02,  1.1426e-01, -2.1968e-01, -9.6342e-02, -2.7733e-02,
          9.9623e-02, -2.3648e-01,  4.3565e-02, -4.6121e-03, -1.9655e-01,
          4.8431e-02,  4.3675e-02,  1.6051e-01,  7.5895e-02, -1.1379e-01,
          4.9420e-02, -2.8200e-01,  2.1536e-01, -1.7395e-01, -7.1819e-02,
          4.2751e-03,  1.3051e-01, -4.7527e-02, -1.6499e-01,  9.5409e-03,
          1.3376e-01, -1.0672e-02,  3.4931e-01,  3.6965e-01,  8.3024e-02,
         -7.3793e-02,  2.4083e-01,  5.6800e-02,  2.1083e-01, -1.0905e-01,
         -1.5902e-01, -1.1473e-01,  2.8838e-01,  7.4191e-02,  6.4179e-02,
          1.4164e-01, -1.0146e-01,  9.1499e-02, -1.8660e-02, -2.6614e-02,
          1.0658e-02,  2.8504e-01,  2.1275e-02,  1.0248e-02, -3.9928e-03,
         -3.7584e-04,  1.2951e-02,  2.5689e-01,  4.2087e-02,  2.0912e-01,
          7.5375e-02, -7.3558e-02, -9.6267e-02, -5.2454e-02, -3.8492e-03,
          9.0253e-03,  2.4067e-01, -2.

In [48]:
input_size

134

In [49]:
#인코더 연산을 클래스로 선언
class Encoder(nn.Module):
    def __init__(self, input_size, hid_dim, n_layer):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.gru = nn.GRU(
            input_size,
            hid_dim,
            n_layer,
            batch_first = True,
            bidirectional = True
        )

        self.fc = nn.Linear(
            hid_dim * 4,
            hid_dim
        )

    def forward(self, x):
        h0 = torch.zeros(self.n_layer*2, x.shape[0], self.hid_dim).cuda().float()
        encoder_output, encoder_hidden = gru(x,h0)

        encoder_hidden = torch.cat((
            encoder_hidden[-4,:,:],
            encoder_hidden[-3,:,:],
            encoder_hidden[-2,:,:],
            encoder_hidden[-1,:,:]
        ), dim = 1)

        encoder_hidden = torch.tanh(self.fc(encoder_hidden))
        return encoder_output, encoder_hidden

In [50]:
encoder_output.shape

torch.Size([1, 240, 1024])

In [51]:
#encoder_output.shape[1] : 인코더 결과의 행의 개수 => 240프레임
src_len = encoder_output.shape[1]

In [52]:
src_len

240

In [53]:
#인코더의 히든 레이어 결과
encoder_hidden

tensor([[-2.6731e-02, -2.8492e-02, -2.2107e-02, -7.6137e-02, -1.6818e-01,
          4.1211e-02,  1.1476e-01, -2.2332e-01, -9.6641e-02, -2.7740e-02,
          9.9955e-02, -2.4105e-01,  4.3593e-02, -4.6121e-03, -1.9914e-01,
          4.8469e-02,  4.3703e-02,  1.6191e-01,  7.6041e-02, -1.1429e-01,
          4.9460e-02, -2.8985e-01,  2.1878e-01, -1.7574e-01, -7.1943e-02,
          4.2752e-03,  1.3126e-01, -4.7563e-02, -1.6651e-01,  9.5412e-03,
          1.3457e-01, -1.0672e-02,  3.6466e-01,  3.8801e-01,  8.3215e-02,
         -7.3927e-02,  2.4566e-01,  5.6862e-02,  2.1404e-01, -1.0948e-01,
         -1.6038e-01, -1.1524e-01,  2.9680e-01,  7.4328e-02,  6.4268e-02,
          1.4260e-01, -1.0181e-01,  9.1755e-02, -1.8662e-02, -2.6620e-02,
          1.0658e-02,  2.9316e-01,  2.1279e-02,  1.0248e-02, -3.9928e-03,
         -3.7584e-04,  1.2952e-02,  2.6277e-01,  4.2112e-02,  2.1225e-01,
          7.5519e-02, -7.3691e-02, -9.6566e-02, -5.2502e-02, -3.8493e-03,
          9.0256e-03,  2.4548e-01, -2.

In [54]:
encoder_hidden.shape

torch.Size([1, 512])

In [55]:
#인코더 히든 레이어의 차원을 1개 더 추가
# unsqueeze(1) : 1번째 차원 행을 1개 더 추가함
encoder_hidden.unsqueeze(1)

tensor([[[-2.6731e-02, -2.8492e-02, -2.2107e-02, -7.6137e-02, -1.6818e-01,
           4.1211e-02,  1.1476e-01, -2.2332e-01, -9.6641e-02, -2.7740e-02,
           9.9955e-02, -2.4105e-01,  4.3593e-02, -4.6121e-03, -1.9914e-01,
           4.8469e-02,  4.3703e-02,  1.6191e-01,  7.6041e-02, -1.1429e-01,
           4.9460e-02, -2.8985e-01,  2.1878e-01, -1.7574e-01, -7.1943e-02,
           4.2752e-03,  1.3126e-01, -4.7563e-02, -1.6651e-01,  9.5412e-03,
           1.3457e-01, -1.0672e-02,  3.6466e-01,  3.8801e-01,  8.3215e-02,
          -7.3927e-02,  2.4566e-01,  5.6862e-02,  2.1404e-01, -1.0948e-01,
          -1.6038e-01, -1.1524e-01,  2.9680e-01,  7.4328e-02,  6.4268e-02,
           1.4260e-01, -1.0181e-01,  9.1755e-02, -1.8662e-02, -2.6620e-02,
           1.0658e-02,  2.9316e-01,  2.1279e-02,  1.0248e-02, -3.9928e-03,
          -3.7584e-04,  1.2952e-02,  2.6277e-01,  4.2112e-02,  2.1225e-01,
           7.5519e-02, -7.3691e-02, -9.6566e-02, -5.2502e-02, -3.8493e-03,
           9.0256e-03,  2

In [56]:
encoder_hidden.unsqueeze(1).shape

torch.Size([1, 1, 512])

In [57]:
#인코더 히든 레이어의 차원을 1개 더 추가
# unsqueeze(1) : 1번쨰 차원 행을 1개 더 추가함

#repeat(1, src_len, 1) : 데이터를 1개의 배열, src_len (240행) 으로 1번 반복
encoder_hidden.unsqueeze(1).repeat(1,src_len,1)

tensor([[[-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801],
         [-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801],
         [-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801],
         ...,
         [-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801],
         [-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801],
         [-0.0267, -0.0285, -0.0221,  ..., -0.0818,  0.0537, -0.1801]]],
       device='cuda:0', grad_fn=<RepeatBackward0>)

In [58]:
encoder_hidden.unsqueeze(1).repeat(1, src_len, 1).shape

torch.Size([1, 240, 512])

In [59]:
#인코더 히든 레이어의 차원을 1개 더 추가
# unsqueeze(1) : 1번째 차원 행을 1개 더 추가함

#repeat(1, src_len, 1) : 데이터를 1개의 배열, src_len (240행) 으로 1번 반복

att_encoder_hidden = encoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

In [60]:
att_encoder_hidden.shape

torch.Size([1, 240, 512])

In [61]:
encoder_output.shape

torch.Size([1, 240, 1024])

In [62]:
#att_encoder_hidden 과 encoder_output 을 1개읠 배열로 함침
#dim = 2 : 열방향으로 합침
torch.cat((att_encoder_hidden, encoder_output), dim = 2)

tensor([[[-0.0267, -0.0285, -0.0221,  ..., -0.1775,  0.1487,  0.0590],
         [-0.0267, -0.0285, -0.0221,  ..., -0.1642,  0.1019, -0.0024],
         [-0.0267, -0.0285, -0.0221,  ..., -0.1341, -0.0537,  0.0109],
         ...,
         [-0.0267, -0.0285, -0.0221,  ..., -0.0007, -0.1417,  0.0567],
         [-0.0267, -0.0285, -0.0221,  ...,  0.0329, -0.2524,  0.0442],
         [-0.0267, -0.0285, -0.0221,  ..., -0.0814, -0.2073, -0.0540]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [63]:
torch.cat((att_encoder_hidden, encoder_output), dim = 2).shape

torch.Size([1, 240, 1536])

In [64]:
#선형 회귀 객체 생성
attn = nn.Linear(
    (HID_DIM * 2) + HID_DIM, #입력데이터의 열의 개수
                             #HID_DIM (512) * 2 + HID_DIM =>
    HID_DIM #선형회귀 결과의 열의 개수
).cuda()

In [65]:
attn

Linear(in_features=1536, out_features=512, bias=True)

In [66]:
#att_encoder_hidden 과 encoder_output 을 1개의 배열로 합침
#dim = 2 : 열방향으로 합침
#torch.cat((att_encoder_hidden, encoder_output), dim = 2)

#attn : 선형회귀 실행
attn (torch.cat((att_encoder_hidden, encoder_output), dim = 2) )

tensor([[[ 0.0089, -0.0350,  0.1939,  ..., -0.1087,  0.0033,  0.0291],
         [ 0.1154, -0.0384,  0.1936,  ..., -0.0893,  0.0022, -0.0118],
         [ 0.0059, -0.0305,  0.1054,  ..., -0.1971, -0.0171, -0.0673],
         ...,
         [-0.0661, -0.0791,  0.2208,  ..., -0.2602, -0.0220, -0.0919],
         [-0.0526, -0.1507,  0.1672,  ..., -0.2168, -0.0561, -0.1017],
         [-0.0170, -0.1074,  0.0828,  ..., -0.1872, -0.0725, -0.0667]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [67]:
#att_encoder_hidden 과 encoder_output 을 1개의 배열로 합침
#dim = 2 : 열방향으로 합침
#torch.cat((att_encoder_hidden, encoder_output), dim = 2)

#attn : 선형회귀 실행
#attn (torch.cat((att_encoder_hidden, encoder_output), dim = 2) )

#torch.tanh : tanh 활성함수 실행
torch.tanh( attn (torch.cat((att_encoder_hidden, encoder_output), dim = 2) ) )

tensor([[[ 0.0089, -0.0350,  0.1915,  ..., -0.1083,  0.0033,  0.0291],
         [ 0.1149, -0.0384,  0.1912,  ..., -0.0890,  0.0022, -0.0118],
         [ 0.0059, -0.0305,  0.1050,  ..., -0.1946, -0.0171, -0.0672],
         ...,
         [-0.0660, -0.0789,  0.2172,  ..., -0.2545, -0.0220, -0.0917],
         [-0.0525, -0.1495,  0.1657,  ..., -0.2134, -0.0560, -0.1013],
         [-0.0170, -0.1070,  0.0826,  ..., -0.1850, -0.0724, -0.0666]]],
       device='cuda:0', grad_fn=<TanhBackward0>)

In [68]:
#att_encoder_hidden 과 encoder_output 을 1개의 배열로 합침
#dim = 2 : 열방향으로 합침
#torch.cat((att_encoder_hidden, encoder_output), dim = 2)

#attn : 선형회귀 실행
#attn (torch.cat((att_encoder_hidden, encoder_output), dim = 2) )

#torch.tanh : tanh 활성함수 실행


energy = torch.tanh( attn (torch.cat((att_encoder_hidden, encoder_output), dim = 2) ) )

In [69]:
energy

tensor([[[ 0.0089, -0.0350,  0.1915,  ..., -0.1083,  0.0033,  0.0291],
         [ 0.1149, -0.0384,  0.1912,  ..., -0.0890,  0.0022, -0.0118],
         [ 0.0059, -0.0305,  0.1050,  ..., -0.1946, -0.0171, -0.0672],
         ...,
         [-0.0660, -0.0789,  0.2172,  ..., -0.2545, -0.0220, -0.0917],
         [-0.0525, -0.1495,  0.1657,  ..., -0.2134, -0.0560, -0.1013],
         [-0.0170, -0.1070,  0.0826,  ..., -0.1850, -0.0724, -0.0666]]],
       device='cuda:0', grad_fn=<TanhBackward0>)

In [70]:
#선형회귀 객체 선언
v = nn.Linear(
    HID_DIM, #입력데이터의 열의 개수 (HID_DIM : 512)
    1, #출력데이터 열의 개수
    bias = False #선형회귀시 더하기 bias사용안함
).cuda()

In [71]:
v(energy)

tensor([[[ 0.0022],
         [-0.0129],
         [ 0.0008],
         [ 0.0263],
         [ 0.0037],
         [ 0.0347],
         [ 0.0330],
         [ 0.0526],
         [ 0.1041],
         [ 0.0542],
         [ 0.0639],
         [ 0.0347],
         [ 0.0766],
         [ 0.0545],
         [ 0.0643],
         [ 0.0289],
         [-0.0072],
         [ 0.0228],
         [ 0.0083],
         [ 0.0314],
         [ 0.0419],
         [ 0.0290],
         [ 0.0205],
         [ 0.0332],
         [ 0.0112],
         [-0.0021],
         [-0.0325],
         [-0.0346],
         [-0.0027],
         [-0.0340],
         [ 0.0015],
         [ 0.0071],
         [-0.0335],
         [ 0.0045],
         [-0.0110],
         [-0.0080],
         [ 0.0413],
         [ 0.0320],
         [ 0.0911],
         [ 0.0741],
         [ 0.0520],
         [-0.0186],
         [-0.0191],
         [-0.0210],
         [ 0.0357],
         [ 0.0684],
         [ 0.0357],
         [ 0.0196],
         [ 0.0118],
         [-0.0041],


In [72]:
v(energy).shape

torch.Size([1, 240, 1])

In [73]:
#선형회귀 실행
#v(energy)

#squeeze(2) : 3차원 배열인 v(energy)
# 차원을 1개 축소시켜 2차원 배열로 변환
# 2번쨰 차원 열을 삭제
v(energy).squeeze(2)

tensor([[ 0.0022, -0.0129,  0.0008,  0.0263,  0.0037,  0.0347,  0.0330,  0.0526,
          0.1041,  0.0542,  0.0639,  0.0347,  0.0766,  0.0545,  0.0643,  0.0289,
         -0.0072,  0.0228,  0.0083,  0.0314,  0.0419,  0.0290,  0.0205,  0.0332,
          0.0112, -0.0021, -0.0325, -0.0346, -0.0027, -0.0340,  0.0015,  0.0071,
         -0.0335,  0.0045, -0.0110, -0.0080,  0.0413,  0.0320,  0.0911,  0.0741,
          0.0520, -0.0186, -0.0191, -0.0210,  0.0357,  0.0684,  0.0357,  0.0196,
          0.0118, -0.0041,  0.0092,  0.0516,  0.0495,  0.0897,  0.0837,  0.0559,
         -0.0250,  0.0130, -0.0282, -0.0134, -0.0111,  0.0249,  0.0029,  0.0363,
          0.0364,  0.0693,  0.0309,  0.0491,  0.0782,  0.0038,  0.0157,  0.0278,
          0.0558,  0.0895,  0.0657,  0.0640,  0.0240,  0.0372,  0.0382,  0.0355,
          0.0412,  0.0334,  0.0348,  0.0258,  0.0152,  0.0204,  0.0016,  0.0653,
         -0.0107, -0.0272, -0.0196,  0.0014,  0.0108, -0.0167,  0.0207,  0.0453,
         -0.0465, -0.0224, -

In [74]:
#선형회귀 실행
#v(energy)

#squeeze(2) : 3차원 배열인 v(energy)
# 차원을 1개 축소시켜 2차원 배열로 변환
# 2번쨰 차원 열을 삭제
v(energy).squeeze(2).shape

torch.Size([1, 240])

In [75]:
#선형회귀 실행
#v(energy)

#squeeze(2) : 3차원 배열인 v(energy)
# 차원을 1개 축소시켜 2차원 배열로 변환
# 2번쨰 차원 열을 삭제

attention_output = v(energy).squeeze(2)

In [76]:
attention_output

tensor([[ 0.0022, -0.0129,  0.0008,  0.0263,  0.0037,  0.0347,  0.0330,  0.0526,
          0.1041,  0.0542,  0.0639,  0.0347,  0.0766,  0.0545,  0.0643,  0.0289,
         -0.0072,  0.0228,  0.0083,  0.0314,  0.0419,  0.0290,  0.0205,  0.0332,
          0.0112, -0.0021, -0.0325, -0.0346, -0.0027, -0.0340,  0.0015,  0.0071,
         -0.0335,  0.0045, -0.0110, -0.0080,  0.0413,  0.0320,  0.0911,  0.0741,
          0.0520, -0.0186, -0.0191, -0.0210,  0.0357,  0.0684,  0.0357,  0.0196,
          0.0118, -0.0041,  0.0092,  0.0516,  0.0495,  0.0897,  0.0837,  0.0559,
         -0.0250,  0.0130, -0.0282, -0.0134, -0.0111,  0.0249,  0.0029,  0.0363,
          0.0364,  0.0693,  0.0309,  0.0491,  0.0782,  0.0038,  0.0157,  0.0278,
          0.0558,  0.0895,  0.0657,  0.0640,  0.0240,  0.0372,  0.0382,  0.0355,
          0.0412,  0.0334,  0.0348,  0.0258,  0.0152,  0.0204,  0.0016,  0.0653,
         -0.0107, -0.0272, -0.0196,  0.0014,  0.0108, -0.0167,  0.0207,  0.0453,
         -0.0465, -0.0224, -

In [77]:
#F.softmax() 활성함수 실행
F.softmax(attention_output, dim = 1)

tensor([[0.0041, 0.0040, 0.0041, 0.0042, 0.0041, 0.0042, 0.0042, 0.0043, 0.0045,
         0.0043, 0.0044, 0.0042, 0.0044, 0.0043, 0.0044, 0.0042, 0.0041, 0.0042,
         0.0041, 0.0042, 0.0043, 0.0042, 0.0042, 0.0042, 0.0041, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0040, 0.0041, 0.0041, 0.0040, 0.0041, 0.0041, 0.0041,
         0.0043, 0.0042, 0.0045, 0.0044, 0.0043, 0.0040, 0.0040, 0.0040, 0.0042,
         0.0044, 0.0042, 0.0042, 0.0041, 0.0041, 0.0041, 0.0043, 0.0043, 0.0045,
         0.0045, 0.0043, 0.0040, 0.0042, 0.0040, 0.0040, 0.0041, 0.0042, 0.0041,
         0.0043, 0.0043, 0.0044, 0.0042, 0.0043, 0.0044, 0.0041, 0.0042, 0.0042,
         0.0043, 0.0045, 0.0044, 0.0044, 0.0042, 0.0043, 0.0043, 0.0042, 0.0043,
         0.0042, 0.0042, 0.0042, 0.0042, 0.0042, 0.0041, 0.0044, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0041, 0.0040, 0.0042, 0.0043, 0.0039, 0.0040, 0.0040,
         0.0038, 0.0040, 0.0041, 0.0042, 0.0042, 0.0041, 0.0044, 0.0043, 0.0042,
         0.0042, 0.0042, 0.0

In [78]:
#F.softmax() 활성함수 실행
attention_output = F.softmax(attention_output, dim = 1)

In [79]:
attention_output

tensor([[0.0041, 0.0040, 0.0041, 0.0042, 0.0041, 0.0042, 0.0042, 0.0043, 0.0045,
         0.0043, 0.0044, 0.0042, 0.0044, 0.0043, 0.0044, 0.0042, 0.0041, 0.0042,
         0.0041, 0.0042, 0.0043, 0.0042, 0.0042, 0.0042, 0.0041, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0040, 0.0041, 0.0041, 0.0040, 0.0041, 0.0041, 0.0041,
         0.0043, 0.0042, 0.0045, 0.0044, 0.0043, 0.0040, 0.0040, 0.0040, 0.0042,
         0.0044, 0.0042, 0.0042, 0.0041, 0.0041, 0.0041, 0.0043, 0.0043, 0.0045,
         0.0045, 0.0043, 0.0040, 0.0042, 0.0040, 0.0040, 0.0041, 0.0042, 0.0041,
         0.0043, 0.0043, 0.0044, 0.0042, 0.0043, 0.0044, 0.0041, 0.0042, 0.0042,
         0.0043, 0.0045, 0.0044, 0.0044, 0.0042, 0.0043, 0.0043, 0.0042, 0.0043,
         0.0042, 0.0042, 0.0042, 0.0042, 0.0042, 0.0041, 0.0044, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0041, 0.0040, 0.0042, 0.0043, 0.0039, 0.0040, 0.0040,
         0.0038, 0.0040, 0.0041, 0.0042, 0.0042, 0.0041, 0.0044, 0.0043, 0.0042,
         0.0042, 0.0042, 0.0

In [80]:
attention_output.shape

torch.Size([1, 240])

In [81]:
#Attention을 클래스로 구현

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()

        self.attn = nn.Linear((hid_dim * 2) + hid_dim, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_output):

        batch_size = encoder_output.shape[0]
        src_len = encoder_output.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_output), dim = 2)))

        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim = 1)

In [82]:
#수어번역결과
y_data

tensor([[  1, 103, 736,  ...,   0,   0,   0],
        [  1, 505, 305,  ...,   0,   0,   0],
        [  1, 484, 448,  ...,   0,   0,   0],
        ...,
        [  1, 403, 143,  ...,   0,   0,   0],
        [  1, 403, 123,  ..., 529, 453,   2],
        [  1, 403, 123,  ...,   0,   0,   0]], dtype=torch.int32)

In [83]:
#수어 번역결과 38 => "부지역"
y_data[0][3:4]

tensor([720], dtype=torch.int32)

In [84]:
input = y_data[0][3:4]

In [85]:
input

tensor([720], dtype=torch.int32)

In [86]:
#input의 차원을 1개 늘려서 2차원으로 변환
#unsqueeze(0) : 0번째 차원 행을 1개 추가
input.unsqueeze(0)

tensor([[720]], dtype=torch.int32)

In [87]:
#input의 차원을 1개 늘려서 2차원으로 변환
#unsqueeze(0) : 0번째 차원 행을 1개 추가
#input.unsqueeze(0)

input = input.unsqueeze(0)

In [88]:
input

tensor([[720]], dtype=torch.int32)

In [89]:
input.shape

torch.Size([1, 1])

In [90]:
OUTPUT_DIM

1078

In [91]:
EMB_DIM

128

In [92]:
#nn.Embedding : 수어 번역 단어를 입력으로 단어의 특성을 잘 표현하는 배열을 생성하는 객체
embedding = nn.Embedding(
    OUTPUT_DIM, #번역할 수어 단어의 종류개수
    EMB_DIM #Embedding 결과 열의 개수
)

In [93]:
embedding

Embedding(1078, 128)

In [94]:
embedding(input)

tensor([[[-0.1419,  1.0665,  1.8103,  0.9215, -0.3699, -0.3985,  0.5305,
          -0.9677,  0.1530, -0.5210,  0.8421, -1.9539, -0.4675,  0.7059,
           0.5563,  1.9950,  0.2280,  0.6944,  0.4038, -1.4784,  0.1829,
          -0.1135, -0.7106,  0.3231,  1.1481,  0.2314, -1.4081, -1.3473,
           0.4470, -1.5788,  0.0310, -0.4250,  0.0878,  0.0913,  1.8632,
           0.7942, -0.8224, -0.2114,  0.4836,  0.3305,  1.0909, -0.6224,
          -1.2750,  1.1412, -0.4788, -1.1304,  1.7502, -0.7665,  0.0974,
           1.2215,  0.4477, -0.1316,  0.4656, -1.6697, -0.9317, -1.6635,
           0.6694,  1.0785, -1.2680, -0.9164,  1.0590, -0.5228, -0.8256,
          -0.4428,  1.2043, -0.3686, -0.9278, -1.0731, -0.8064,  0.2029,
           0.0444,  0.4846,  0.5846,  1.0107,  0.6770,  1.2714,  0.6559,
           0.1976,  0.5915,  0.3368, -0.5658,  2.0398,  1.3629,  2.0164,
           0.2637,  0.9488, -0.5347,  0.6231, -2.0867, -1.0632,  0.3333,
          -0.8872,  0.4860, -0.9380, -0.3168,  0.52

In [95]:
embedding(input).shape

torch.Size([1, 1, 128])

In [96]:
#드롭아웃을 실행할 객체 생성
dropout = nn.Dropout(0.5)

In [97]:
##nn.Embedding : 수어 번역 단어를 입력으로 단어의 특성을 잘 표현하는 배열을 생성하는 Embedding객체에 dropout 실행
dropout(embedding(input))

tensor([[[-0.2838,  2.1331,  3.6205,  0.0000, -0.0000, -0.7969,  0.0000,
          -0.0000,  0.3059, -0.0000,  1.6841, -0.0000, -0.0000,  0.0000,
           0.0000,  3.9900,  0.4560,  0.0000,  0.0000, -2.9568,  0.0000,
          -0.0000, -1.4211,  0.0000,  2.2962,  0.0000, -2.8163, -2.6946,
           0.8940, -3.1577,  0.0000, -0.8500,  0.0000,  0.1826,  3.7263,
           0.0000, -1.6448, -0.0000,  0.9672,  0.0000,  2.1818, -0.0000,
          -0.0000,  0.0000, -0.9576, -0.0000,  3.5003, -1.5331,  0.1948,
           0.0000,  0.0000, -0.0000,  0.0000, -0.0000, -0.0000, -3.3269,
           0.0000,  0.0000, -0.0000, -1.8327,  2.1179, -0.0000, -1.6512,
          -0.0000,  0.0000, -0.0000, -1.8556, -2.1463, -1.6128,  0.4058,
           0.0000,  0.9693,  0.0000,  0.0000,  1.3541,  0.0000,  0.0000,
           0.0000,  1.1830,  0.0000, -0.0000,  0.0000,  2.7259,  0.0000,
           0.0000,  1.8977, -1.0695,  0.0000, -4.1733, -2.1265,  0.0000,
          -1.7744,  0.0000, -0.0000, -0.0000,  1.04

In [98]:
embedded = dropout(embedding(input))

In [99]:
embedded

tensor([[[-0.0000,  2.1331,  3.6205,  1.8431, -0.7399, -0.7969,  1.0610,
          -1.9354,  0.0000, -0.0000,  0.0000, -0.0000, -0.9350,  0.0000,
           1.1125,  0.0000,  0.4560,  0.0000,  0.0000, -0.0000,  0.3657,
          -0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -2.8163, -0.0000,
           0.8940, -0.0000,  0.0619, -0.8500,  0.0000,  0.1826,  0.0000,
           1.5884, -1.6448, -0.4228,  0.0000,  0.6610,  2.1818, -1.2447,
          -0.0000,  0.0000, -0.9576, -2.2609,  0.0000, -1.5331,  0.1948,
           0.0000,  0.0000, -0.2633,  0.0000, -3.3394, -1.8633, -0.0000,
           1.3387,  2.1570, -0.0000, -0.0000,  0.0000, -1.0456, -0.0000,
          -0.0000,  2.4087, -0.7371, -1.8556, -0.0000, -0.0000,  0.4058,
           0.0000,  0.0000,  1.1692,  0.0000,  1.3541,  2.5428,  0.0000,
           0.3952,  1.1830,  0.6736, -0.0000,  0.0000,  0.0000,  0.0000,
           0.5274,  1.8977, -0.0000,  1.2462, -0.0000, -2.1265,  0.6666,
          -1.7744,  0.9720, -0.0000, -0.0000,  1.04

In [100]:
embedded.shape

torch.Size([1, 1, 128])

In [101]:
#Attention 결과가 저장된 변수
attention_output

tensor([[0.0041, 0.0040, 0.0041, 0.0042, 0.0041, 0.0042, 0.0042, 0.0043, 0.0045,
         0.0043, 0.0044, 0.0042, 0.0044, 0.0043, 0.0044, 0.0042, 0.0041, 0.0042,
         0.0041, 0.0042, 0.0043, 0.0042, 0.0042, 0.0042, 0.0041, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0040, 0.0041, 0.0041, 0.0040, 0.0041, 0.0041, 0.0041,
         0.0043, 0.0042, 0.0045, 0.0044, 0.0043, 0.0040, 0.0040, 0.0040, 0.0042,
         0.0044, 0.0042, 0.0042, 0.0041, 0.0041, 0.0041, 0.0043, 0.0043, 0.0045,
         0.0045, 0.0043, 0.0040, 0.0042, 0.0040, 0.0040, 0.0041, 0.0042, 0.0041,
         0.0043, 0.0043, 0.0044, 0.0042, 0.0043, 0.0044, 0.0041, 0.0042, 0.0042,
         0.0043, 0.0045, 0.0044, 0.0044, 0.0042, 0.0043, 0.0043, 0.0042, 0.0043,
         0.0042, 0.0042, 0.0042, 0.0042, 0.0042, 0.0041, 0.0044, 0.0041, 0.0040,
         0.0040, 0.0041, 0.0041, 0.0040, 0.0042, 0.0043, 0.0039, 0.0040, 0.0040,
         0.0038, 0.0040, 0.0041, 0.0042, 0.0042, 0.0041, 0.0044, 0.0043, 0.0042,
         0.0042, 0.0042, 0.0

In [102]:
attention_output.shape

torch.Size([1, 240])

In [103]:
#Attention 결과가 저장된 변수 attention_output 배열차원을 1추가
#unsqueeze(1) : 1번째 차원 추가
attention_output.unsqueeze(1)

tensor([[[0.0041, 0.0040, 0.0041, 0.0042, 0.0041, 0.0042, 0.0042, 0.0043,
          0.0045, 0.0043, 0.0044, 0.0042, 0.0044, 0.0043, 0.0044, 0.0042,
          0.0041, 0.0042, 0.0041, 0.0042, 0.0043, 0.0042, 0.0042, 0.0042,
          0.0041, 0.0041, 0.0040, 0.0040, 0.0041, 0.0040, 0.0041, 0.0041,
          0.0040, 0.0041, 0.0041, 0.0041, 0.0043, 0.0042, 0.0045, 0.0044,
          0.0043, 0.0040, 0.0040, 0.0040, 0.0042, 0.0044, 0.0042, 0.0042,
          0.0041, 0.0041, 0.0041, 0.0043, 0.0043, 0.0045, 0.0045, 0.0043,
          0.0040, 0.0042, 0.0040, 0.0040, 0.0041, 0.0042, 0.0041, 0.0043,
          0.0043, 0.0044, 0.0042, 0.0043, 0.0044, 0.0041, 0.0042, 0.0042,
          0.0043, 0.0045, 0.0044, 0.0044, 0.0042, 0.0043, 0.0043, 0.0042,
          0.0043, 0.0042, 0.0042, 0.0042, 0.0042, 0.0042, 0.0041, 0.0044,
          0.0041, 0.0040, 0.0040, 0.0041, 0.0041, 0.0040, 0.0042, 0.0043,
          0.0039, 0.0040, 0.0040, 0.0038, 0.0040, 0.0041, 0.0042, 0.0042,
          0.0041, 0.0044, 0.0043, 0.00

In [104]:
#Attention 결과가 저장된 변수 attention_output 배열차원을 1추가
#unsqueeze(1) : 1번째 차원 추가
attention_output.unsqueeze(1).shape

torch.Size([1, 1, 240])

In [105]:
#Attention 결과가 저장된 변수 attention_output 배열차원을 1추가
#unsqueeze(1) : 1번째 차원 추가
#attention_output.unsqueeze(1)
attention_output = attention_output.unsqueeze(1)

In [106]:
attention_output.shape

torch.Size([1, 1, 240])

In [107]:
#encoder_output : 인코더 결과
encoder_output.shape

torch.Size([1, 240, 1024])

In [108]:
#attention_output (어텐션 결과) encoder_output(인코더 결과)
# 의 곱을 계산
torch.bmm(attention_output, encoder_output)

tensor([[[-0.0623,  0.1425,  0.2584,  ..., -0.1821,  0.1533, -0.0247]]],
       device='cuda:0', grad_fn=<BmmBackward0>)

In [109]:
#attention_output(어텐션결과) encoder_output(인코더결과)
#의 곱을 계산
#torch.bmm(attention_output, encoder_output)
torch.bmm(attention_output, encoder_output).shape

torch.Size([1, 1, 1024])

In [110]:
weighted = torch.bmm(attention_output, encoder_output)

In [111]:
weighted

tensor([[[-0.0623,  0.1425,  0.2584,  ..., -0.1821,  0.1533, -0.0247]]],
       device='cuda:0', grad_fn=<BmmBackward0>)

In [112]:
weighted.shape

torch.Size([1, 1, 1024])

In [113]:
#weighted.permute(1,0,2): weighted의 배열의 차원을 1, 0, 2로 변환
weighted.permute(1,0,2)

tensor([[[-0.0623,  0.1425,  0.2584,  ..., -0.1821,  0.1533, -0.0247]]],
       device='cuda:0', grad_fn=<PermuteBackward0>)

In [114]:
weighted = weighted.permute(1,0,2)

In [115]:
weighted

tensor([[[-0.0623,  0.1425,  0.2584,  ..., -0.1821,  0.1533, -0.0247]]],
       device='cuda:0', grad_fn=<PermuteBackward0>)

In [116]:
weighted.shape

torch.Size([1, 1, 1024])

In [117]:
##입력한 단어 38 => "부지역"
#특성을 저장한 배열 embedded

#attention_output(어텐션결과) encoder_output(인코더결과)
#의 곱을 계산 한 weighted

#torch.cat((embedded.cuda(), weighted.cuda()), dim= 2)
#embedded와 weighted를 하나로 합침
#dim =2 : 배열의 2번째 차원에서 합침
rnn_input = torch.cat((embedded.cuda(), weighted.cuda()), dim = 2)

In [118]:
rnn_input

tensor([[[-0.0000,  2.1331,  3.6205,  ..., -0.1821,  0.1533, -0.0247]]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [119]:
rnn_input.shape

torch.Size([1, 1, 1152])

In [120]:
#encoder_hidden: 인코더의 실행 결과
encoder_hidden.shape

torch.Size([1, 512])

In [121]:
#.unsqueeze(0): 차원을 1증가
#                0번째 차원을 1 증가
encoder_hidden.unsqueeze(0).shape

torch.Size([1, 1, 512])

In [122]:
#GRU 연산 실행 할 객체
gru = nn.GRU(
             HID_DIM*2+EMB_DIM, #입력값의 열의 개수
                               #HID_DIM(512) * 2 + EMB_DIM(128) => 1152
             HID_DIM         #hidden layer의 열의 개수 HID_DIM(512) 출력 512 할다다
            ).cuda()

In [123]:
rnn_input.shape

torch.Size([1, 1, 1152])

In [124]:
encoder_hidden.shape

torch.Size([1, 512])

In [125]:
#GRU 연산

#rnn_input:
##입력한 단어 38 => "부지역"
#특성을 저장한 배열 embedded

#attention_output(어텐션 결과) encoder_output(인코더결과)
#의 곱을 계산 한 weighted
#하나로 합침

#encoder_hidden : 인코더의 실행 결과
#.unsqueeze(0): 차원을 1증가
#               0번째 차원을 증가

gru(
    rnn_input.cuda(),
    encoder_hidden.unsqueeze(0).cuda()
)


##65페이지 여기서부터 다시치기 

(tensor([[[ 0.0531,  0.1032, -0.3259, -0.0329,  0.2102, -0.0819,  0.2782,
           -0.1939, -0.0209,  0.0204,  0.0541, -0.0774, -0.0147,  0.0439,
            0.0974,  0.2558,  0.1534, -0.0048, -0.1775,  0.1940,  0.2559,
            0.0911,  0.4602, -0.1321, -0.0504, -0.0969,  0.0923, -0.0790,
           -0.0063, -0.0842,  0.1760, -0.0056,  0.2708, -0.0958,  0.2160,
           -0.1291,  0.0010,  0.0935,  0.2566,  0.1014, -0.0209,  0.0225,
           -0.1249,  0.0580,  0.0675, -0.1330, -0.0669,  0.3336, -0.0116,
           -0.0761, -0.4173,  0.0891, -0.0179,  0.0597, -0.1879,  0.1096,
           -0.1793,  0.0509, -0.0942,  0.4694, -0.2836,  0.2470,  0.1813,
            0.1659,  0.1959, -0.1338,  0.1581, -0.1622,  0.0559, -0.4297,
           -0.0129,  0.1603, -0.0344, -0.1248,  0.4458, -0.0053, -0.0601,
            0.0449, -0.1083,  0.1231, -0.0264,  0.2023,  0.1130, -0.1210,
            0.0123, -0.1203,  0.0847, -0.0948, -0.1133,  0.2529,  0.3957,
            0.0554,  0.1320,  0.0210, 

In [126]:
#GRU 연산
#rnn_input:
## 입력한 단어 38 => "부지역"
#특성을 저장한 배열 embedded
#attention_output(어텐션결과) encoder_output(인코더결과)
#의 곱을 계산한 weighted 하나로 합침

#encoder_hidden: 인코더의 실행 결과
#.unsqueeze(0) : 차원을 1증가
#                0번째 차원을 1 증가
decoder_out, decoder_hidden = gru(rnn_input.cuda(), encoder_hidden.unsqueeze(0).cuda())

In [127]:
decoder_out

tensor([[[ 0.0531,  0.1032, -0.3259, -0.0329,  0.2102, -0.0819,  0.2782,
          -0.1939, -0.0209,  0.0204,  0.0541, -0.0774, -0.0147,  0.0439,
           0.0974,  0.2558,  0.1534, -0.0048, -0.1775,  0.1940,  0.2559,
           0.0911,  0.4602, -0.1321, -0.0504, -0.0969,  0.0923, -0.0790,
          -0.0063, -0.0842,  0.1760, -0.0056,  0.2708, -0.0958,  0.2160,
          -0.1291,  0.0010,  0.0935,  0.2566,  0.1014, -0.0209,  0.0225,
          -0.1249,  0.0580,  0.0675, -0.1330, -0.0669,  0.3336, -0.0116,
          -0.0761, -0.4173,  0.0891, -0.0179,  0.0597, -0.1879,  0.1096,
          -0.1793,  0.0509, -0.0942,  0.4694, -0.2836,  0.2470,  0.1813,
           0.1659,  0.1959, -0.1338,  0.1581, -0.1622,  0.0559, -0.4297,
          -0.0129,  0.1603, -0.0344, -0.1248,  0.4458, -0.0053, -0.0601,
           0.0449, -0.1083,  0.1231, -0.0264,  0.2023,  0.1130, -0.1210,
           0.0123, -0.1203,  0.0847, -0.0948, -0.1133,  0.2529,  0.3957,
           0.0554,  0.1320,  0.0210, -0.1022,  0.19

In [128]:
decoder_hidden

tensor([[[ 0.0531,  0.1032, -0.3259, -0.0329,  0.2102, -0.0819,  0.2782,
          -0.1939, -0.0209,  0.0204,  0.0541, -0.0774, -0.0147,  0.0439,
           0.0974,  0.2558,  0.1534, -0.0048, -0.1775,  0.1940,  0.2559,
           0.0911,  0.4602, -0.1321, -0.0504, -0.0969,  0.0923, -0.0790,
          -0.0063, -0.0842,  0.1760, -0.0056,  0.2708, -0.0958,  0.2160,
          -0.1291,  0.0010,  0.0935,  0.2566,  0.1014, -0.0209,  0.0225,
          -0.1249,  0.0580,  0.0675, -0.1330, -0.0669,  0.3336, -0.0116,
          -0.0761, -0.4173,  0.0891, -0.0179,  0.0597, -0.1879,  0.1096,
          -0.1793,  0.0509, -0.0942,  0.4694, -0.2836,  0.2470,  0.1813,
           0.1659,  0.1959, -0.1338,  0.1581, -0.1622,  0.0559, -0.4297,
          -0.0129,  0.1603, -0.0344, -0.1248,  0.4458, -0.0053, -0.0601,
           0.0449, -0.1083,  0.1231, -0.0264,  0.2023,  0.1130, -0.1210,
           0.0123, -0.1203,  0.0847, -0.0948, -0.1133,  0.2529,  0.3957,
           0.0554,  0.1320,  0.0210, -0.1022,  0.19

In [129]:
embedded.shape

torch.Size([1, 1, 128])

In [130]:
##입력한 단어 38 => "부지역"
#특성을 저장한 배열 embedded
#emnedded.squeeze(0) : embedded 의 차원을 삭제
#squeeze(0) : 0 번째 차원 삭제
embedded.squeeze(0).shape

torch.Size([1, 128])

In [131]:
embedded = embedded.squeeze(0)

In [132]:
decoder_out = decoder_out.squeeze(0)

In [133]:
decoder_out.shape

torch.Size([1, 512])

In [134]:
#weighted:
##attention_output(어텐션결과) encoder_output(인코더결과) 곱
weighted.shape

torch.Size([1, 1, 1024])

In [135]:
#weighted.squeeze(0): weighted의 차원을 삭제
#squeeze(0) : 0번째 차원 삭제
weighted = weighted.squeeze(0)

In [136]:
weighted.shape

torch.Size([1, 1024])

In [137]:
#decoder_out: 디코더 연산 결과
#weighted : attention_output(어텐션 결과) encoder_output(인코더 결과) 곱
#embedded: ##입력한 단어 38 => "부지역" 특성을 저장한 배열
#torch.cat(): 하나로 합침
#dim = 1 : 배열의 1번째 차원에서 합침
torch.cat((decoder_out.cuda(), weighted.cuda(), embedded.cuda()), dim = 1)

tensor([[ 0.0531,  0.1032, -0.3259,  ..., -2.6939,  0.0000, -0.0000]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [138]:
decoder_out = torch.cat((decoder_out.cuda(), weighted.cuda(), embedded.cuda()), dim=1)

In [139]:
decoder_out.shape

torch.Size([1, 1664])

In [140]:
#nn.Linear: 선형회귀를 실행 할 객체 생성
fc_out = nn.Linear(
                    (HID_DIM*2) + HID_DIM+ EMB_DIM, #입력값의 열 개수 
                                                    #(HID_DIM(512)*2) + HID_DIM(512) + EMB_DIM(128) = 1664
                    OUTPUT_DIM #선형 회귀 결과의 열개수 OUTPUT_DIM(162)
).cuda()

In [141]:
#decoder_out : 디코더 연산 결과
#weighted : attention_output (어텐션 결과) encoder_output(인코더결과) 곱
#embedded : ##입력한 단어 38 => "부지역" 특성을 저장한 배열
#을 1개로 통합한 decoder_out
#선형회귀 실행
fc_out(decoder_out)

tensor([[ 0.2303, -0.1374,  0.3837,  ...,  0.0708, -0.0787, -0.1628]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [142]:
prediction = fc_out(decoder_out)

In [143]:
#예측결과
prediction

tensor([[ 0.2303, -0.1374,  0.3837,  ...,  0.0708, -0.0787, -0.1628]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [144]:
#디코더 연산을 클래스로 구현
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layer, attention, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.attention = attention
        self.n_layer = n_layer
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.gru = nn.GRU(hid_dim*2+emb_dim,hid_dim)
        self.fc_out = nn.Linear((hid_dim*2)+hid_dim+emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, encoder_hidden, encoder_output):

        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))

        attention_output = self.attention(encoder_hidden, encoder_output)
        attention_output = attention_output.unsqueeze(1)

        weighted = torch.bmm(attention_output, encoder_output)

        weighted = weighted.permute(1,0,2)

        rnn_input = torch.cat((embedded, weighted), dim = 2)

        decoder_output, decoder_hidden = self.gru(rnn_input, encoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        decoder_output = decoder_output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((decoder_output, weighted, embedded), dim = 1))

        return prediction,decoder_hidden.squeeze(0)

In [145]:
#SequenceToSequence 구현

class GRU_AT_Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    #딥러닝 학습시 호출되는 함수
    #src : 수어 keypoint
    #trg : 수어 스크립트 문장이 저장
    def forward(self, src, trg, teacher_forcing_ration = 0.5):
        #trg.shape[0] : trg에 저장된 문장의 개수 (16)
        batch_size = trg.shape[0]

        #trg.shape[1] : 수어 스크립트 단어의 개수 (30)
        trg_len = trg.shape[1]

        #decoder.output_dim : 수어 스크립트를 구성하는 단어의 개수 (162)
        trg_vocab_size = self.decoder.output_dim

        #torch.zeros(trg_len, batch_size, trg_vocab_size): 0으로 초기화된 배열 생성
                                                        # 배열 shape[trg_len(30), batch_size(16), trg_vocab_size(162)]
        output = torch.zeros(trg_len, batch_size, trg_vocab_size).cuda()

        #self.encoder(src) : 인코더 연산 실행
        encoder_output, hidden = self.encoder(src)

        #trg[:,0] :수어스크립트 trg 0번째 칸 리턴 <START>
        input = trg[:,0]

        for t in range(1, trg_len):
            #self.decoder(input, hidden, encoder_output) :디코더 연산 실행
            decoder_output, hidden = self.decoder(input, hidden, encoder_output)

            output [t] = decoder_output        #decoder_output: 디코더 연산 결과

            #random.random() < teacher_forcing_ration : 0~1 사이 난수를 생성해서 난수가 0.5 미만이면 난수 리턴
                                                     # 난수가 0.5이상이면 None 리턴

            teacher_force = random.random() < teacher_forcing_ration

            #decoder_output.argmax(1) : decoder_output에서 가장 확률이 높은 단어 대입
            top1 = decoder_output.argmax(1)

            #trg[:,t] if teacher_force : teacher_force 가 None이 아니면 수어스크립트가 저장된 trg[:,t] 리턴
            #  else top1:                teacher_force 가 None이면 decoder의 예측값 top1리턴
            input = trg[:,t] if teacher_force else top1

        return output

In [146]:
input_size

134

In [147]:
enc = Encoder(input_size, HID_DIM, N_LAYER).to(device)
enc

Encoder(
  (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=2048, out_features=512, bias=True)
)

In [148]:
att = Attention(HID_DIM).to(device)
att

Attention(
  (attn): Linear(in_features=1536, out_features=512, bias=True)
  (v): Linear(in_features=512, out_features=1, bias=False)
)

In [149]:
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYER, att, DEC_DROPOUT).to(device)
dec

Decoder(
  (attention): Attention(
    (attn): Linear(in_features=1536, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
  (embedding): Embedding(1078, 128)
  (gru): GRU(1152, 512)
  (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [150]:
model = GRU_AT_Seq2Seq(enc,dec,device).to(device)
model

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [151]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [152]:
model.apply(init_weights)

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [153]:
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [154]:
def train(model, dataloader, OUTPUT_DIM,optimizer, criterion, clip):
    #학습시작
    model.train()

    epoch_loss = 0
    
    #dataloader : keypoint와 수어스크립트 단어를 리턴하는 객체
    #input: keypoint
    #target: 수어스크립트 단어
    for i, (input, target) in enumerate(dataloader):

        src = input #keypoint 대입
        trg = target #수어스크립트 단어 대입
        
        #torch.cuda.is_available() : GRU 사용 가능하면 True
        if torch.cuda.is_available():
            model.cuda() #model을 gru에 저장

            #src.cuda(): src를 GPU에 저장
            #.float(): 실수타입으로 변환
            src = src.cuda().float()

            #trg.cuda(): trg를 GPU에 저장
            #.long(): long타입으로 변환
            trg = trg.cuda().long()

        #0으로 초기화
        optimizer.zero_grad()
        #model 실행
        output = model(src, trg)

        #output[0] => <START>  입력의 예측결과 저장
        #output[1] : 수어 스크립트 저장

        #view(-1, OUTPUT_DIM): 배열 차원을 OUTPUT_DIM(162)열로 변환
                                # 행은 생략(-1)
        output = output[1:].view(-1, OUTPUT_DIM)
        #torch.transpose(trg,0,1): trg 배열 차원의 0번째(행) 1번째(열) 서로 바꾸기
        trg = torch.transpose(trg,0,1)

        #trg[1:].contiguous() : 배열에 저장된 요소들의 메모리 주소가 서로 인접하도록 설정
        #view(-1) :1 차원 배열로 변환
        trg = trg[1:].contiguous().view(-1)

        #criterion(output, trg) : output(예측한 수어스크립트) trg(실제 수어스크립트)의 차이를 계산
        loss = criterion(output, trg)

        #역전파 실행
        loss.backward()

        #output(예측한 수어스크립트) trg(실제 수어스크립트) 의 loss가 1이상이면 #loss를 clip(1)로 치환해서 역전파 실행
        torch.nn.utils.clip_grad_norm_(model.parameters(),clip)

        #역전파 결과로 매개변수 수정
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [155]:
#학습진행

best_loss = float('inf')

train_loss_Ls = []

for epoch in tqdm(range(N_EPOCH)):
    #train 함수 호출
    train_loss = train(model, train_dataloader, OUTPUT_DIM, optimizer, criterion, CLIP)

    #train_loss를 리스트에 추가
    train_loss_ls.append(train_loss)

    #train_loss가 best_loss 미만
    if train_loss < best_loss:
        #train_loss를 대입
        best_loss = train_loss
        #학습모델 저장
        torch.sava(model.state_dict(), f'{model_sava_path}{save_model_name}')

    #cache 삭제
    torch.cuda.empty_cache()
    print(f'Epoch:{epoch + 1:02}')
    print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print("="*100)

  0%|          | 0/1000 [00:00<?, ?it/s]

RuntimeError: input.size(-1) must be equal to input_size. Expected 1152, got 134